In [2]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd

In [2]:
dataTrain=pd.read_csv("../data/newtrain.csv")
dataTest=pd.read_csv("../data/newtest.csv")
print(dataTrain.head())
print(dataTrain.shape)

   Unnamed: 0  round  venue   gf   ga   xg  xga  poss  formation    sh  ...  \
0           0     33      0  0.0  0.0  1.4  0.8  49.0          4  11.0  ...   
1           1     35      1  2.0  0.0  2.4  0.4  64.0          4  25.0  ...   
2           2     37      0  5.0  2.0  3.5  1.0  67.0          4  30.0  ...   
3           3      2      0  1.0  3.0  0.3  1.8  43.0          4   6.0  ...   
4           4      3      1  2.0  0.0  2.1  0.7  43.0          4  14.0  ...   

   ga_rolling  sh_rolling  sot_rolling  dist_rolling  fk_rolling  pk_rolling  \
0    2.333333   17.333333     6.333333     17.300000    0.333333         0.0   
1    1.333333   12.000000     3.666667     16.566667    0.333333         0.0   
2    0.000000   17.666667     6.333333     17.166667    1.000000         0.0   
3    0.666667   22.000000     8.000000     17.500000    1.333333         0.0   
4    1.666667   20.333333     8.333333     18.500000    1.666667         0.0   

   pkatt_rolling  home_code  target_away  ta

In [3]:
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]

In [4]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          1806 non-null   int64  
 1   venue          1806 non-null   int64  
 2   gf             1806 non-null   float64
 3   ga             1806 non-null   float64
 4   xg             1806 non-null   float64
 5   xga            1806 non-null   float64
 6   poss           1806 non-null   float64
 7   formation      1806 non-null   int64  
 8   sh             1806 non-null   float64
 9   sot            1806 non-null   float64
 10  dist           1806 non-null   float64
 11  fk             1806 non-null   float64
 12  pk             1806 non-null   float64
 13  pkatt          1806 non-null   float64
 14  venue_code     1806 non-null   int64  
 15  opp_code       1806 non-null   int64  
 16  hour           1806 non-null   int64  
 17  day_code       1806 non-null   int64  
 18  gf_rolli

In [5]:
dataTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          609 non-null    int64  
 1   venue          609 non-null    int64  
 2   gf             609 non-null    float64
 3   ga             609 non-null    float64
 4   xg             609 non-null    float64
 5   xga            609 non-null    float64
 6   poss           609 non-null    float64
 7   formation      609 non-null    int64  
 8   sh             609 non-null    float64
 9   sot            609 non-null    float64
 10  dist           609 non-null    float64
 11  fk             609 non-null    float64
 12  pk             609 non-null    float64
 13  pkatt          609 non-null    float64
 14  venue_code     609 non-null    int64  
 15  opp_code       609 non-null    int64  
 16  hour           609 non-null    int64  
 17  day_code       609 non-null    int64  
 18  gf_rolling

In [6]:
# Data Training
train_data = dataTrain.iloc[:, :-2].values
train_labels = dataTrain.iloc[:, -2:].values

# Data Testing
test_data = dataTest.iloc[:, :-2].values
test_labels = dataTest.iloc[:, -2:].values

In [7]:
# Normalisasi Data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Pembentukan Data Sequence
sequence_length = 10  # Panjang sequence yang akan digunakan
num_features = train_data.shape[1]

In [8]:
def create_sequences(data, labels, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(labels[i+sequence_length])
    return np.array(X), np.array(y)


In [9]:
X_train, y_train = create_sequences(train_data, train_labels, sequence_length)
X_test, y_test = create_sequences(test_data, test_labels, sequence_length)

In [10]:
# Pembangunan Model LSTM
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, num_features)))
model.add(Dense(units=2, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [11]:
# Pelatihan Model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Sesuaikan dengan jumlah epoch dan ukuran batch yang optimal

Epoch 1/10
57/57 [==============================] - 4s 65ms/step - loss: 0.6560 - accuracy: 0.5874
Epoch 2/10
57/57 [==============================] - 3s 61ms/step - loss: 0.6428 - accuracy: 0.6214
Epoch 3/10
57/57 [==============================] - 3s 60ms/step - loss: 0.6396 - accuracy: 0.6102
Epoch 4/10
57/57 [==============================] - 3s 60ms/step - loss: 0.6421 - accuracy: 0.6242
Epoch 5/10
57/57 [==============================] - 4s 61ms/step - loss: 0.6378 - accuracy: 0.6303
Epoch 6/10
57/57 [==============================] - 3s 60ms/step - loss: 0.6364 - accuracy: 0.6303
Epoch 7/10
57/57 [==============================] - 3s 60ms/step - loss: 0.6342 - accuracy: 0.6269
Epoch 8/10
57/57 [==============================] - 3s 59ms/step - loss: 0.6332 - accuracy: 0.6325
Epoch 9/10
57/57 [==============================] - 4s 62ms/step - loss: 0.6313 - accuracy: 0.6297
Epoch 10/10
57/57 [==============================] - 4s 63ms/step - loss: 0.6339 - accuracy: 0.6347


In [12]:
model.save('../model/model2.h5')

In [13]:
from tensorflow.keras.models import load_model
model = load_model('../model/model2.h5')

In [27]:
new_data = np.array([[0, 0, 2, 0, 1, 1.2, 44, 15, 10, 2, 14.6, 1, 0, 0, 0, 6, 20, 4, 1.666666667, 2, 15, 5, 20.7, 0, 0.3333333333, 0.3333333333, 0]])